In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg
import math
from scipy import stats

In [2]:
# Initial inventory at Retailer, wholesaler, distributor, and manufaturer, assuming manufacturer has unlimited capacity (very high)
init_inv = np.array([200, 200, 200])
t_horizon = 26
consumer_demand = np.random.normal(100, 5, t_horizon) # Normal
#consumer_demand = np.random.random(t_horizon) * 100 # Random
consumer_demand = np.trunc(consumer_demand)
consumer_demand

array([ 97.,  97.,  94.,  98., 100.,  99.,  93.,  95., 101.,  93., 100.,
       103., 104.,  93.,  97.,  94.,  98.,  94.,  96., 101., 100., 103.,
       102.,  96., 103., 104.])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

plt.plot(np.arange(1, 27, step=1), consumer_demand)
ax.set_xlabel("Time")
ax.set_ylabel("Order Quantity")
ax.set_title("Consumer Demand")
plt.show()

In [ ]:
# At time t=1
o_t_equals_one_retailer = consumer_demand[0]
o_t_equals_one_wholesaler = o_t_equals_one_retailer
o_t_equals_one_distributor = o_t_equals_one_wholesaler
o_t_equals_one_manufacturer = o_t_equals_one_distributor
o_t_equals_one = np.array([o_t_equals_one_retailer, o_t_equals_one_wholesaler, o_t_equals_one_distributor, o_t_equals_one_manufacturer])
o_t_equals_one

In [ ]:
# Define an order function
def order_func1(d_t, d_t_minus_one, o_t_minus_one):
    o_t_minus_one_retailer = o_t_minus_one[0]
    o_t_minus_one_wholesaler = o_t_minus_one[1]
    o_t_minus_one_distributor = o_t_minus_one[2]
    
    o_t_retailer = d_t + (d_t - d_t_minus_one)
    o_t_wholesaler = o_t_retailer + (o_t_retailer - o_t_minus_one_retailer)
    o_t_distributor = o_t_wholesaler + (o_t_wholesaler - o_t_minus_one_wholesaler)
    #o_t_manufacturer = o_t_distributor + (o_t_distributor - o_t_minus_one_distributor)
    o_t = np.array([o_t_retailer, o_t_wholesaler, o_t_distributor])
    
    return o_t

def order_func2(d_t, d_t_minus_one, o_t_minus_one):
    o_t_minus_one_retailer = o_t_minus_one[0]
    o_t_minus_one_wholesaler = o_t_minus_one[1]
    o_t_minus_one_distributor = o_t_minus_one[2]
    
    o_t_retailer = (d_t + d_t_minus_one) / 2
    o_t_wholesaler = (o_t_retailer + o_t_minus_one_retailer) / 2
    o_t_distributor = (o_t_wholesaler + o_t_minus_one_wholesaler) / 2
    #o_t_manufacturer = o_t_distributor + (o_t_distributor - o_t_minus_one_distributor)
    o_t = np.array([o_t_retailer, o_t_wholesaler, o_t_distributor])
    
    return o_t

def order_func3(d_t):
    
    o_t_retailer = d_t
    o_t_wholesaler = o_t_retailer
    o_t_distributor = o_t_wholesaler
    #o_t_manufacturer = o_t_distributor + (o_t_distributor - o_t_minus_one_distributor)
    o_t = np.array([o_t_retailer, o_t_wholesaler, o_t_distributor])
    
    return o_t


def back_order_func(d_t, v_0):
    
    o_t_retailer = d_t
    o_t_wholesaler = o_t_retailer + np.maximum(0, o_t_retailer - v_0[1])
    o_t_distributor = o_t_wholesaler + np.maximum(0, o_t_wholesaler - v_0[2])
    #o_t_manufacturer = o_t_distributor + (o_t_distributor - o_t_minus_one_distributor)
    o_t = np.array([o_t_retailer, o_t_wholesaler, o_t_distributor])
    
    return o_t


# Define a function to round up to the nearest 10
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [4]:
val0 = 0
val1 = 5
np.maximum(0, 5)

5

In [ ]:
print(order_func1(consumer_demand[1], consumer_demand[0], o_t_equals_one))
print(order_func2(consumer_demand[1], consumer_demand[0], o_t_equals_one))
print(order_func3(consumer_demand[1]))

In [ ]:
consumer_demand

In [ ]:
order = np.zeros(3 * t_horizon).reshape((-1, 3)) # order array for each participant over time horizon

for t in range(1, consumer_demand.size+1):
    if t == 1:
        o_t = o_t_equals_one[:3]
    else:
        #o_t = order_func1(consumer_demand[t-1], consumer_demand[t-2], o_t_minus_one)
        o_t = order_func2(consumer_demand[t-1], consumer_demand[t-2], o_t_minus_one)
        #o_t = order_func3(consumer_demand[t-1])
    o_t = np.maximum(o_t, np.zeros(3))
    order[t-1, :] = o_t
    o_t_minus_one = o_t    

In [ ]:
order_including_consumer = np.block([consumer_demand.reshape((-1, 1)), order])

In [ ]:
order_including_consumer

In [ ]:
order_min = np.min(order_including_consumer)
order_max = np.max(order_including_consumer)

order_min = roundup(order_min)
order_max = roundup(order_max)

order_min, order_max

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

x_ticks = np.arange(1, t_horizon+1)
plt.plot(x_ticks, order_including_consumer[:, 0], label="Consumer")
plt.plot(x_ticks, order_including_consumer[:, 1], label="Retailer")
plt.plot(x_ticks, order_including_consumer[:, 2], label="Wholesaler")
plt.plot(x_ticks, order_including_consumer[:, 3], label="Distributor")
ax.set_xlabel("Time")
ax.set_ylabel("Order Quantity")
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs[0, 0].plot(x_ticks, order_including_consumer[:, 0])
axs[0, 0].set_title("Consumer Demand")
axs[0, 1].plot(x_ticks, order_including_consumer[:, 1], 'tab:orange')
axs[0, 1].set_title("Retailer's Order to Wholesaler")
axs[1, 0].plot(x_ticks, order_including_consumer[:, 2], 'tab:green')
axs[1, 0].set_title("Wholesaler's Order to Distributor")
axs[1, 1].plot(x_ticks, order_including_consumer[:, 3], 'tab:red')
axs[1, 1].set_title("Distributor's Order to Manufacturer")

for ax in axs.flat:
    ax.set(xlabel="Time", ylabel="Order Quantity")
    ax.set_ylim(80, order_max)
    
#plt.suptitle("Increasing Variability of Orders up in the Shoe Supply Chain", fontsize=16, fontweight="bold")
plt.suptitle("Decreasing Variability of Orders up in the Shoe Supply Chain", fontsize=16, fontweight="bold")
fig.tight_layout()

#plt.show()
#plt.savefig("bullwhip.png", dpi=150)
plt.savefig("reverse_bullwhip.png", dpi=150)

In [ ]:
np.std(order_including_consumer, axis=0)